In [1]:
import os
import datetime
from collections import OrderedDict

import pandas as pd

import tika
tika.initVM()
from tika import parser

In [2]:
import os
entries = os.listdir('Updated')

In [3]:
len(entries)

183

In [4]:
# Cutting edge for fast speed
cut='About this data.\n\n'
checkreg='Mobility trends for places of residence' #divide country and region pages

In [5]:
#clean the data
def clean_process(mess):
    mess=mess.split('\n')
    messls=[]
    for i in mess:
        if(i!=''):
            messls.append(i)
    return messls        

In [6]:
#Extract data for country
def extract_data(mess):
    value=[]
    for i in range(len(mess)):
        if (mess[i] in 'Retail & recreation '):
           value.append(mess[i+1][:-1])
        if (mess[i] in 'Grocery & pharmacy '):
           value.append(mess[i+1][:-1])
        if (mess[i] in 'Parks '):
           value.append(mess[i+1][:-1])     
        if (mess[i] in 'Transit stations '):
           value.append(mess[i+1][:-1])
        if (mess[i] in 'Workplaces '):
           value.append(mess[i+1][:-1])
        if (mess[i] in 'Residential '):
           value.append(mess[i+1][:-1])
        if (len(value)==6):
            return value

In [7]:
#extract data for reason
def extract_dataR(mess):
    value=[]
    hold=[]
    for i in range(len(mess)):
        if (mess[i] in 'Retail & recreation '):
           value.append(mess[i-1])
           value.append(mess[i+1][:3])
        if (mess[i] in 'Grocery & pharmacy '):
            value.append(mess[i+1][:3])
        if (mess[i] in 'Parks '):
            value.append(mess[i+1][:3])     
        if (mess[i] in 'Transit stations '):
            value.append(mess[i+1][:3])
        if (mess[i] in 'Workplace '):
           value.append(mess[i+1][:3])
        if (mess[i] in 'Residential '):
            value.append(mess[i+1][:3])
        if (len(value)==7):
            hold.append(value)
            value=[]
    return(hold)
       

In [8]:
#remove unwanted file
entries.remove('.DS_Store')

In [51]:
#PDF to CSV
for i in entries:
    parsed = parser.from_file('Updated/'+i)
    text=parsed["content"]
    Name=[]
    Retail=[]
    Groc=[]
    Park=[]
    Transit=[]
    Workplace=[]
    Res=[]
    
    text=text.split(checkreg)
    country=text[0].split(cut)[1]
    country=clean_process(country)
    output=extract_data(country)
    
    Name.append(i.split("_")[1])
    Retail.append(output[0])
    Groc.append(output[1])
    Park.append(output[2])
    Transit.append(output[3])
    Workplace.append(output[4])
    Res.append(output[5])
    
    if(len(text)==2):
        reg=text[1]
        reg=clean_process(reg)
        out=extract_dataR(reg)
        for j in out:
            Name.append(j[0])
            Retail.append(j[1])
            Groc.append(j[2])
            Park.append(j[3])
            Transit.append(j[4])
            Workplace.append(j[5])
            Res.append(j[6])
    
    df=pd.DataFrame()
    df["Name"]=Name
    df["Retail"]=Retail
    df["Groc"]=Groc
    df["Transit"]=Transit
    df["Workplace"]=Workplace
    df["Res"]=Res
    df["Park"]=Park
    df.to_csv("CSV/"+i.split("_")[1]+i.split("_")[2]+'.csv')